In [75]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
#from pytorch_tabnet.tab_model import TabNetModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
import warnings
import shap
import optuna
import torch
from sklearn import svm
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tqdm.auto import tqdm
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from pycaret.regression import *
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import ExtraTreeRegressor
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

In [76]:
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', None)

In [88]:
train_data = pd.read_csv("./train.csv")

In [89]:
train_data['YEAR'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.year
train_data['MONTH'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.month
train_data['DAY'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.day
train_data['HOUR'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.hour
train_data['MINUTE'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.minute
train_data['DAY_OF_WEEK'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.day_name()

train_data = train_data.drop(['MONTH', 'DAY', 'YEAR', 'MINUTE', 'HOUR'], axis = 1)

In [90]:
colls = []
for coll in train_data :
    if 'X' in coll and train_data[coll].isna().sum() == 598 :
        colls.append(coll)
print(len(colls))

train_data.drop(colls, inplace = True, axis = 1)

82


In [91]:
train_data.loc[train_data.PRODUCT_CODE == 'A_31','PRODUCT_CODE'] = 1
train_data.loc[train_data.PRODUCT_CODE == 'T_31','PRODUCT_CODE'] = 2
train_data.loc[train_data.PRODUCT_CODE == 'O_31','PRODUCT_CODE'] = 3

train_data.loc[train_data.DAY_OF_WEEK == 'Monday','DAY_OF_WEEK'] = 1
train_data.loc[train_data.DAY_OF_WEEK == 'Tuesday','DAY_OF_WEEK'] = 2
train_data.loc[train_data.DAY_OF_WEEK == 'Wednesday','DAY_OF_WEEK'] = 3
train_data.loc[train_data.DAY_OF_WEEK == 'Thursday','DAY_OF_WEEK'] = 4
train_data.loc[train_data.DAY_OF_WEEK == 'Friday','DAY_OF_WEEK'] = 5
train_data.loc[train_data.DAY_OF_WEEK == 'Saturday','DAY_OF_WEEK'] = 6
train_data.loc[train_data.DAY_OF_WEEK == 'Sunday','DAY_OF_WEEK'] = 7

train_data.drop(['PRODUCT_ID', 'TIMESTAMP'], axis = 1, inplace = True)
train_data

Y_Class  Y_Quality     LINE PRODUCT_CODE   X_1   X_2  X_3   X_4   X_5  \
0          1   0.533433  T050304            1   NaN   NaN  NaN   NaN   NaN   
1          2   0.541819  T050307            1   NaN   NaN  NaN   NaN   NaN   
2          1   0.531267  T050304            1   NaN   NaN  NaN   NaN   NaN   
3          2   0.537325  T050307            1   NaN   NaN  NaN   NaN   NaN   
4          1   0.531590  T050304            1   NaN   NaN  NaN   NaN   NaN   
..       ...        ...      ...          ...   ...   ...  ...   ...   ...   
593        1   0.526546  T100306            2   2.0  95.0  0.0  45.0  10.0   
594        0   0.524022  T050304            1   NaN   NaN  NaN   NaN   NaN   
595        0   0.521289  T050304            1   NaN   NaN  NaN   NaN   NaN   
596        1   0.531375  T100304            3  40.0  94.0  0.0  45.0  11.0   
597        1   0.533702  T100306            3  21.0  87.0  0.0  45.0  10.0   

     X_6   X_7   X_8   X_9  X_10   X_11   X_12   X_13  X_14  X_15   X_16  \
0    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
1    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
2    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
3    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
4    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
..   ...   ...   ...   ...   ...    ...    ...    ...   ...   ...    ...   
593  0.0  50.0  10.0  52.0   2.0  468.9  473.9  459.2   0.0   0.0  247.5   
594  NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
595  NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
596  0.0  45.0  10.0  31.0   2.0  505.8  510.9  497.1   0.0   0.0  247.5   
597  0.0  61.0  10.0  52.0   2.0  470.6  474.9  462.9   0.0   0.0  247.5   

      X_17   X_18   X_19   X_20   X_21   X_22  X_23  X_24  X_25   X_26   X_27  \
0      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
1      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
2      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
3      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
4      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
..     ...    ...    ...    ...    ...    ...   ...   ...   ...    ...    ...   
593  248.0  247.3  250.0  527.0  538.0  522.0  0.19  0.19  0.19  155.0  155.0   
594    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
595    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
596  248.1  247.3  250.0  489.0  498.0  485.0  0.19  0.19  0.19  155.0  155.0   
597  248.0  247.3  250.0  525.0  534.0  520.0  0.19  0.20  0.19  155.0  155.0   

      X_28   X_29   X_30   X_31   X_32   X_33   X_34   X_35   X_36   X_37  \
0      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
593  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0  155.0   
594    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
595    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
596  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0  155.0   
597  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0  155.0   

      X_38  X_39  X_40  X_41  X_42  X_43  X_44  X_45  X_46  X_47  X_48  X_49  \
0      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2      NaN   NaN 

In [92]:
T050304 = train_data[train_data['LINE'] == 'T050304']
T050307 = train_data[train_data['LINE'] == 'T050307']
T010305 = train_data[train_data['LINE'] == 'T010305']
T010306 = train_data[train_data['LINE'] == 'T010306']
T100304 = train_data[train_data['LINE'] == 'T100304']
T100306 = train_data[train_data['LINE'] == 'T100306']

T050304.drop(['LINE'], axis = 1, inplace = True)
T050307.drop(['LINE'], axis = 1, inplace = True)
T010305.drop(['LINE'], axis = 1, inplace = True)
T010306.drop(['LINE'], axis = 1, inplace = True)
T100304.drop(['LINE'], axis = 1, inplace = True)
T100306.drop(['LINE'], axis = 1, inplace = True)

In [93]:
T050304_col = []
for col in T050304 :
    if 'X' in col and T050304[col].isna().sum() == len(T050304) :
        T050304_col.append(col)
print(len(T050304_col))

T050307_col = []
for col in T050307 :
    if 'X' in col and T050307[col].isna().sum() == len(T050307) :
        T050307_col.append(col)
print(len(T050307_col))

T010305_col = []
for col in T010305 :
    if 'X' in col and T010305[col].isna().sum() == len(T010305) :
        T010305_col.append(col)
print(len(T010305_col))

T010306_col = []
for col in T010306 :
    if 'X' in col and T010306[col].isna().sum() == len(T010306) :
        T010306_col.append(col)
print(len(T010306_col))

T100304_col = []
for col in T100304 :
    if 'X' in col and T100304[col].isna().sum() == len(T100304) :
        T100304_col.append(col)
print(len(T100304_col))

T100306_col = []
for col in T100306 :
    if 'X' in col and T100306[col].isna().sum() == len(T100306) :
        T100306_col.append(col)
print(len(T100306_col))

825
818
1907
1907
2122
2122


In [94]:
T050304.drop(T050304_col, inplace = True, axis = 1)
T050307.drop(T050307_col, inplace = True, axis = 1)
T010305.drop(T010305_col, inplace = True, axis = 1)
T010306.drop(T010306_col, inplace = True, axis = 1)
T100304.drop(T100304_col, inplace = True, axis = 1)
T100306.drop(T100306_col, inplace = True, axis = 1)

In [95]:
T051 = list(set(T050307_col)-set(T050304_col))
T052 = list(set(T050304_col)-set(T050307_col))
T5drop = T051 + T052

T011 = list(set(T010305_col)-set(T010306_col))
T012 = list(set(T010306_col)-set(T010305_col))
T0drop = T011 + T012

T101 = list(set(T100304_col)-set(T100306_col))
T102 = list(set(T100306_col)-set(T100304_col))
T1drop = T101 + T102

In [96]:
T05 = pd.concat([T050304, T050307])
T01 = pd.concat([T010305, T010306])
T10 = pd.concat([T100304, T100306])

T05.drop(T5drop, inplace = True, axis = 1)
T01.drop(T0drop, inplace = True, axis = 1)
T10.drop(T1drop, inplace = True, axis = 1)

#T05 = T05.fillna(T05.mean())
#T01 = T01.fillna(T01.mean())
#T10 = T10.fillna(T10.mean())

T05 = T05.fillna(0)
T01 = T01.fillna(0)
T10 = T10.fillna(0)

In [97]:
T05_cols = []
for col in T05 :
    if 'X' in col and T05[col].std() == 0 :
        T05_cols.append(col)
print(len(T05_cols))

T01_cols = []
for col in T01 :
    if 'X' in col and T01[col].std() == 0 :
        T01_cols.append(col)
print(len(T01_cols))

T10_cols = []
for col in T10 :
    if 'X' in col and T10[col].std() == 0 :
        T10_cols.append(col)
print(len(T10_cols))

184
144
91


In [98]:
T05.drop(T05_cols, inplace = True, axis = 1)
T01.drop(T01_cols, inplace = True, axis = 1)
T10.drop(T10_cols, inplace = True, axis = 1)

# T05

In [99]:
T05_class = T05['Y_Class']
T05_target = T05['Y_Quality']
T05_data = T05.drop(['Y_Class', 'Y_Quality'], axis = 1)

In [100]:
T_05c = T05_data.columns
T_05scaler = RobustScaler()
T_05scaled = T_05scaler.fit_transform(T05_data)
T05_data = pd.DataFrame(T_05scaled, columns = T_05c)

T05_y_class = pd.DataFrame(T05_target, columns = ['Y_Quality'])
T05_idx = T05_y_class.index
T05_y_class = T05_y_class.reset_index()
T05_data = pd.concat([T05_data, T05_y_class], axis = 1)
T05_data = T05_data.set_index('index')
T05_data.drop(['Y_Quality'], axis = 1, inplace = True)
T05_data

PRODUCT_CODE  X_132  X_133  X_134  X_135     X_136     X_137     X_138  \
index                                                                           
0               0.0   -1.0   -1.0   -1.0    0.0 -0.199797 -0.201826 -0.208864   
2               0.0   -1.0   -1.0   -1.0    0.0 -0.199797 -0.201826 -0.208864   
4               0.0   -1.0   -1.0   -2.0    0.0 -0.199797 -0.201826 -0.208864   
6               0.0   -1.0   -1.0   -1.0    0.0 -0.199797 -0.201826 -0.208864   
8               0.0   -1.0   -1.0   -1.0    0.0 -0.199797 -0.201826 -0.208864   
...             ...    ...    ...    ...    ...       ...       ...       ...   
549             0.0   -1.0   -1.0   -1.0    0.0 -0.497972 -0.500000 -0.502292   
550             0.0   -1.0   -1.0   -1.0    0.0 -0.495943 -0.497972 -0.500255   
551             0.0   -1.0   -1.0   -1.0    0.0 -0.491886 -0.493915 -0.496179   
552             0.0   -1.0   -1.0   -1.0    0.0 -0.491886 -0.493915 -0.496179   
555             0.0   -1.0   -1.0   -1.0    0.0 -0.485801 -0.487830 -0.490066   

          X_139     X_140     X_141  X_143  X_144  X_145     X_146     X_148  \
index                                                                          
0     -0.024793 -0.052154  0.023438    0.0   -1.0    0.0 -0.003454  0.000000   
2      0.024793 -0.029478  0.315104    0.0    0.0    0.0  0.563040 -0.074074   
4     -0.008264 -0.043084 -0.049479    0.0   -1.0    0.0 -0.348877  0.000000   
6      0.002755 -0.043084 -1.195312    0.0   -2.0    0.0 -4.113990  0.000000   
8     -0.002755 -0.047619  0.263021    0.0    0.0    0.0  0.514680  0.000000   
...         ...       ...       ...    ...    ...    ...       ...       ...   
549   -1.049587 -0.882086 -0.898437    0.0    0.0    0.0  0.373057  1.037037   
550   -1.049587 -0.891156 -0.929687    0.0   -1.0    0.0  0.158895  1.037037   
551   -1.044077 -0.886621 -0.695312    0.0    0.0    0.0  0.618307  1.037037   
552   -1.033058 -0.877551 -0.919271    0.0    0.0    0.0  0.158895  1.037037   
555   -1.082645 -0.877551 -1.263021    0.0   -1.0    0.0 -0.873921  1.111111   

          X_149   X_150  X_151  X_152  X_153  X_155  X_156  X_157  X_159  \
index                                                                      
0      0.072727  0.0625   -1.0    0.0    0.0    0.0    0.0    0.0   -1.0   
2      0.000000  0.0000   -1.0    0.0   -1.0    0.0    0.0    0.0    0.0   
4      0.000000  0.0625    0.0    0.0   -1.0    0.0    0.1    0.1   -1.0   
6      0.000000  0.0000    0.0    0.1    0.0   -1.0    0.0    0.0    0.0   
8      0.000000  0.0625    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...         ...     ...    ...    ...    ...    ...    ...    ...    ...   
549    1.163636  0.8750   -1.0    0.0   -2.0    0.0    0.0    0.0   -1.0   
550    1.163636  0.8750    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
551    1.163636  0.8750   -1.0    0.0   -1.0   -1.0    0.0    0.0   -1.0   
552    1.163636  0.8750   -1.0   -0.1   -1.0    0.0    0.0   -0.1   -1.0   
555    1.236364  0.8750    0.0    0.0    0.0   -1.0    0.0    0.0    0.0   

       X_160  X_161  X_163  X_164  X_165  X_167  X_168  X_169  X_171  X_172  \
index                                                                         
0        0.0    0.0   -0.1    0.0   -0.5   -1.0    0.0    0.0    0.0    0.0   
2        0.0    0.0   -0.1    0.0   -0.5   -1.0    0.0    0.0   -1.0    0.0   
4        0.0    0.0    0.0    1.0    0.5   -1.0    0.0    0.0   -1.0    0.0   
6        0.0    0.0    0.0    0.0   -0.5    0.0    0.0    0.0   -1.0    0.0   
8        0.0    0.0    0.0    1.0    0.5    0.0    0.0    0.0    0.0    0.0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
549      0.0    0.0   -0.1    0.0   -0.5    0.0    0.0    0.0    0.0    0.0   
550      0.0    0.0    0.0    1.0    0.5    0.0    0.0    0.0   -1.0    0.0   
551      0.0    0.0    0.0    0.0    0.5    0.0    0.1    0.0    0.0    0.0   
552      0.1    0.0   -0.1    0.0   -0.5    0.0    0.0

In [101]:
#13
setup_rgs = setup(data = T05_data, target = T05_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,4
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(120, 1775)"
4,Transformed data shape,"(120, 1775)"
5,Transformed train set shape,"(84, 1775)"
6,Transformed test set shape,"(36, 1775)"
7,Numeric features,1774
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0046,0.0000,0.0062,0.5920,0.0040,0.0086,0.2440
catboost,CatBoost Regressor,0.0050,0.0001,0.0063,0.6415,0.0041,0.0093,13.6170
knn,K Neighbors Regressor,0.0079,0.0001,0.0107,-0.1817,0.0069,0.0147,0.0420
lasso,Lasso Regression,0.0083,0.0001,0.0105,-0.1075,0.0069,0.0154,0.2390
en,Elastic Net,0.0082,0.0001,0.0106,-0.1176,0.0069,0.0153,0.0400
llar,Lasso Least Angle Regression,0.0082,0.0001,0.0105,-0.1016,0.0068,0.0154,0.0400
xgboost,Extreme Gradient Boosting,0.0053,0.0001,0.0069,0.4891,0.0045,0.0098,0.1710
dt,Decision Tree Regressor,0.0066,0.0001,0.0091,-0.0203,0.0059,0.0124,0.0430
rf,Random Forest Regressor,0.0055,0.0001,0.0070,0.5027,0.0045,0.0102,0.2830
ada,AdaBoost Regressor,0.0054,0.0001,0.0068,0.5465,0.0044,0.0101,0.1320


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [102]:
T05_total_models = []

T05_model_lightgbm = create_model('lightgbm', fold = 5)
T05_total_models.append(T05_model_lightgbm)

T05_model_rf = create_model('rf', fold = 5)
T05_total_models.append(T05_model_rf)

T05_model_gbr = create_model('gbr', fold = 5)
T05_total_models.append(T05_model_gbr)

T05_model_et = create_model('et', fold = 5)
T05_total_models.append(T05_model_et)

T05_model_catboost = create_model('catboost', fold = 5)
T05_total_models.append(T05_model_catboost)

T05_model_xgb = create_model('xgboost', fold = 5)
T05_total_models.append(T05_model_xgb)

T05_model_ada = create_model('ada', fold = 5)

T05_total_models.append(T05_model_ada)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0087,0.0002,0.0124,0.4187,0.0080,0.0161
1,0.0043,0.0000,0.0064,0.5686,0.0042,0.0080
2,0.0044,0.0000,0.0054,0.6445,0.0035,0.0082
3,0.0066,0.0001,0.0090,0.1046,0.0059,0.0124
4,0.0042,0.0000,0.0055,0.6568,0.0036,0.0079
Mean,0.0056,0.0001,0.0077,0.4786,0.0050,0.0105
Std,0.0018,0.0000,0.0027,0.2053,0.0017,0.0033


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0093,0.0002,0.0124,0.4209,0.0080,0.0171
1,0.0043,0.0000,0.0052,0.7184,0.0034,0.0081
2,0.0040,0.0000,0.0048,0.7252,0.0031,0.0076
3,0.0055,0.0001,0.0077,0.3403,0.0050,0.0103
4,0.0056,0.0001,0.0073,0.3933,0.0047,0.0105
Mean,0.0057,0.0001,0.0075,0.5196,0.0048,0.0107
Std,0.0019,0.0000,0.0027,0.1671,0.0017,0.0034


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0079,0.0001,0.0112,0.5266,0.0072,0.0146
1,0.0039,0.0000,0.0052,0.7123,0.0034,0.0073
2,0.0035,0.0000,0.0043,0.7813,0.0028,0.0066
3,0.0055,0.0001,0.0075,0.3754,0.0049,0.0104
4,0.0060,0.0001,0.0086,0.1423,0.0056,0.0114
Mean,0.0054,0.0001,0.0074,0.5076,0.0048,0.0100
Std,0.0016,0.0000,0.0025,0.2316,0.0016,0.0029


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0075,0.0001,0.0106,0.5714,0.0069,0.0139
1,0.0033,0.0000,0.0042,0.8116,0.0028,0.0061
2,0.0052,0.0000,0.0069,0.4260,0.0045,0.0100
3,0.0049,0.0000,0.0068,0.4879,0.0044,0.0093
4,0.0046,0.0001,0.0072,0.4067,0.0046,0.0086
Mean,0.0051,0.0001,0.0071,0.5407,0.0046,0.0096
Std,0.0014,0.0000,0.0020,0.1472,0.0013,0.0025


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0073,0.0001,0.0106,0.5736,0.0068,0.0134
1,0.0041,0.0000,0.0054,0.6977,0.0035,0.0076
2,0.0042,0.0000,0.0048,0.7239,0.0031,0.0079
3,0.0059,0.0001,0.0079,0.3091,0.0051,0.0111
4,0.0060,0.0001,0.0081,0.2533,0.0052,0.0114
Mean,0.0055,0.0001,0.0073,0.5115,0.0048,0.0103
Std,0.0012,0.0000,0.0021,0.1956,0.0013,0.0022


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [103]:
T05_model_lightgbm = tune_model(T05_model_lightgbm, fold=5, choose_better = True)

T05_model_rf = tune_model(T05_model_rf, fold=5, choose_better = True)

T05_model_gbr = tune_model(T05_model_gbr, fold=5, choose_better = True)

T05_model_et = tune_model(T05_model_et, fold=5, choose_better = True)

T05_model_catboost = tune_model(T05_model_catboost, fold=5, choose_better = True)

T05_model_xgb = tune_model(T05_model_xgb, fold=5, choose_better = True)

T05_model_ada = tune_model(T05_model_ada, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0119,0.0003,0.0163,-0.0088,0.0106,0.0220
1,0.0068,0.0001,0.0100,-0.0538,0.0065,0.0127
2,0.0073,0.0001,0.0092,-0.0336,0.0060,0.0139
3,0.0077,0.0001,0.0096,-0.0169,0.0062,0.0144
4,0.0077,0.0001,0.0094,-0.0100,0.0061,0.0145
Mean,0.0083,0.0001,0.0109,-0.0247,0.0071,0.0155
Std,0.0018,0.0001,0.0027,0.0171,0.0017,0.0033


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0104,0.0002,0.0137,0.2869,0.0089,0.0193
1,0.0043,0.0001,0.0073,0.4484,0.0047,0.0079
2,0.0058,0.0001,0.0071,0.3915,0.0046,0.0109
3,0.0062,0.0001,0.0085,0.1981,0.0055,0.0116
4,0.0042,0.0000,0.0059,0.5936,0.0039,0.0079
Mean,0.0062,0.0001,0.0085,0.3837,0.0055,0.0115
Std,0.0023,0.0001,0.0027,0.1357,0.0018,0.0042


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0085,0.0001,0.0116,0.4927,0.0075,0.0157
1,0.0046,0.0000,0.0053,0.7048,0.0035,0.0085
2,0.0039,0.0000,0.0046,0.7399,0.0030,0.0074
3,0.0055,0.0001,0.0074,0.3915,0.0048,0.0103
4,0.0055,0.0001,0.0077,0.3150,0.0050,0.0103
Mean,0.0056,0.0001,0.0073,0.5288,0.0048,0.0104
Std,0.0016,0.0000,0.0024,0.1682,0.0016,0.0028


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [104]:
T05_tuned_models = [T05_model_lightgbm, T05_model_rf, T05_model_gbr, T05_model_et, T05_model_catboost, T05_model_xgb, T05_model_ada]

for model in T05_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0057,0.0001,0.0075,0.3715,0.0049,0.0107


,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.0,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,0.4,0.526346,0.534627
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,0.0,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.6,0.531590,0.538058
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.0,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,-0.2,0.527398,0.529219
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,1.0,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,0.2,0.537733,0.534449
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,1.0,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,-0.2,0.516711,0.525321
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,0.0,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,-0.2,0.528027,0.526165
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,0.0,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.6,0.531503,0.535638
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,0.0,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.4,0.535405,0.528538
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,1.0,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.6,0.538049,0.537446
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,0.0,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,-0.2,0.519451,0.524440


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0053,0.0000,0.0067,0.4976,0.0044,0.0100


,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.0,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,0.4,0.526346,0.531961
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,0.0,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.6,0.531590,0.532365
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.0,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,-0.2,0.527398,0.526264
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,1.0,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,0.2,0.537733,0.533781
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,1.0,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,-0.2,0.516711,0.526038
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,0.0,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,-0.2,0.528027,0.526092
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,0.0,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.6,0.531503,0.532211
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,0.0,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.4,0.535405,0.525760
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,1.0,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.6,0.538049,0.537896
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,0.0,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,-0.2,0.519451,0.526254


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0056,0.0000,0.0070,0.4509,0.0046,0.0107


...

,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.0,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,0.4,0.526346,0.532805
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,0.0,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.6,0.531590,0.533643
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.0,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,-0.2,0.527398,0.528222
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,1.0,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,0.2,0.537733,0.533343
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,1.0,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,-0.2,0.516711,0.526774
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,0.0,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,-0.2,0.528027,0.527261
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,0.0,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.6,0.531503,0.533333
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,0.0,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.4,0.535405,0.525818
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,1.0,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.6,0.538049,0.537889
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,0.0,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,-0.2,0.519451,0.525488


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.0050,0.0000,0.0064,0.5366,0.0042,0.0094


,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.0,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,0.4,0.526346,0.532652
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,0.0,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.6,0.531590,0.535887
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.0,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,-0.2,0.527398,0.529514
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,1.0,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,0.2,0.537733,0.532930
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,1.0,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,-0.2,0.516711,0.527540
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,0.0,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,-0.2,0.528027,0.527513
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,0.0,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.6,0.531503,0.532369
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,0.0,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.4,0.535405,0.526431
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,1.0,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.6,0.538049,0.535022
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,0.0,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,-0.2,0.519451,0.524756


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0051,0.0000,0.0067,0.5069,0.0043,0.0097


,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.0,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,0.4,0.526346,0.529675
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,0.0,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.6,0.531590,0.532529
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.0,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,-0.2,0.527398,0.526470
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,1.0,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,0.2,0.537733,0.531181
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,1.0,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,-0.2,0.516711,0.527150
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,0.0,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,-0.2,0.528027,0.525764
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,0.0,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.6,0.531503,0.532293
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,0.0,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.4,0.535405,0.526420
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,1.0,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.6,0.538049,0.536673
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,0.0,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,-0.2,0.519451,0.526725


In [105]:
T05_final_regression_model = finalize_model(T05_model_et)

T05_prediction = predict_model(T05_final_regression_model, data=T05_data)
T05_prediction

T05_datas = T05_prediction['prediction_label']
T05_datas = pd.DataFrame(T05_datas.values, columns = ['Y_Quality'])
T05_datas

,Y_Quality
0,0.533433
1,0.531267
2,0.531590
3,0.533665
4,0.531821
...,...
115,0.533711
116,0.533695
117,0.525271
118,0.529806


In [106]:
T05_xtrain, T05_xtest, T05_ytrain, T05_ytest = train_test_split(T05_datas, T05_class, test_size = 0.1, random_state = 4)

In [107]:
scores = []
for thisC in [*range(1,100)]:
    svc=SVC(kernel='rbf',C=thisC, decision_function_shape = 'ovo')
    model=svc.fit(T05_xtrain,T05_ytrain)
    scoreTrain=model.score(T05_xtrain,T05_ytrain)
    scoreTest=model.score(T05_xtest,T05_ytest)
    print("선형 SVM : C:{}, training score:{:2f}, test score:{:2f}".format
          (thisC,scoreTrain, scoreTest))
    scores.append([scoreTrain, scoreTest])

선형 SVM : C:1, training score:0.972222, test score:0.916667
선형 SVM : C:2, training score:0.962963, test score:0.916667
선형 SVM : C:3, training score:0.981481, test score:0.916667
선형 SVM : C:4, training score:0.981481, test score:0.916667
선형 SVM : C:5, training score:0.972222, test score:0.916667
선형 SVM : C:6, training score:0.972222, test score:0.916667
선형 SVM : C:7, training score:0.972222, test score:0.916667
선형 SVM : C:8, training score:0.972222, test score:0.916667
선형 SVM : C:9, training score:0.972222, test score:0.916667
선형 SVM : C:10, training score:0.972222, test score:0.916667
선형 SVM : C:11, training score:0.972222, test score:0.916667
선형 SVM : C:12, training score:0.972222, test score:0.916667
선형 SVM : C:13, training score:0.972222, test score:0.916667
선형 SVM : C:14, training score:0.972222, test score:0.916667
선형 SVM : C:15, training score:0.972222, test score:0.916667
선형 SVM : C:16, training score:0.972222, test score:0.916667
선형 SVM : C:17, training score:0.972222, test scor

In [108]:
T05_svm = svm.SVC(kernel = 'rbf', decision_function_shape = 'ovo', C = 45)
T05_svm.fit(T05_xtrain, T05_ytrain)
T05_svm.predict(T05_xtest)

array([1, 1, 1, 2, 0, 1, 1, 2, 2, 0, 2, 2], dtype=int64)

# T01

In [109]:
T01_class = T01['Y_Class']
T01_target = T01['Y_Quality']
T01_data = T01.drop(['Y_Class', 'Y_Quality'], axis = 1)

In [110]:
T_01c = T01_data.columns
T_01scaler = RobustScaler()
T_01scaled = T_01scaler.fit_transform(T01_data)
T01_data = pd.DataFrame(T_01scaled, columns = T_01c)

T01_y_class = pd.DataFrame(T01_target, columns = ['Y_Quality'])
T01_idx = T01_y_class.index
T01_y_class = T01_y_class.reset_index()
T01_data = pd.concat([T01_data, T01_y_class], axis = 1)
T01_data = T01_data.set_index('index')
T01_data.drop(['Y_Quality'], axis = 1, inplace = True)
T01_data

,PRODUCT_CODE,X_246,X_247,X_248,X_250,X_251,X_253,X_254,X_256,X_257,...,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,DAY_OF_WEEK
index,,,,,,,,,,,,,,,,,,,,,
28,0.0,0.0,-0.25,-0.054348,0.0,1.011236,-1.0,-1.0,-0.145455,-0.198381,...,-0.763158,-0.390876,0.000000,-0.053030,0.321839,0.041096,-1.9,-1.452964,-5.0,0.666667
33,0.0,0.0,1.25,-0.054348,0.0,1.101124,-1.0,-1.0,-0.131818,-0.137652,...,-0.331579,-0.287500,-0.222798,-0.530303,-0.217241,-0.109589,-1.9,-1.500816,-5.0,-1.000000
39,0.0,0.0,1.25,-0.054348,0.0,1.146067,0.0,-1.0,-0.072727,-0.097166,...,-0.163158,-1.014237,-0.621762,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-0.666667
40,0.0,0.0,1.25,-0.054348,0.0,1.179775,-1.0,-1.0,0.031818,0.000000,...,-0.489474,-0.542842,-1.191710,0.689394,0.663885,0.595890,-1.9,-1.604342,-5.0,-0.666667
41,0.0,0.0,-0.75,-0.054348,0.0,1.179775,-1.0,-1.0,0.027273,0.000000,...,-0.931579,-1.034646,-1.492228,-0.022727,-0.040517,0.205479,-1.9,-1.524742,-8.5,-0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,0.0,0.0,0.75,1.108696,0.0,0.000000,0.0,0.0,0.068182,0.048583,...,1.121053,0.285900,0.093264,0.045455,-0.109329,-0.390411,-0.9,-0.482871,-5.0,-1.000000
501,0.0,1.0,1.25,1.293478,0.0,0.000000,0.0,0.0,0.295455,0.246964,...,0.515789,0.550583,-0.134715,0.371212,0.939778,1.041096,0.1,0.344334,1.0,0.000000
523,0.0,0.0,-0.25,1.423913,1.0,0.000000,0.0,0.0,0.459091,0.421053,...,0.200000,0.387683,0.424870,0.492424,0.902874,0.876712,0.1,0.264525,0.0,0.666667


In [111]:
from pycaret.regression import *
setup_rgs = setup(data = T01_data, target = T01_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,4
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(129, 737)"
4,Transformed data shape,"(129, 737)"
5,Transformed train set shape,"(90, 737)"
6,Transformed test set shape,"(39, 737)"
7,Numeric features,736
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,0.0051,0.0000,0.0068,0.1652,0.0044,0.0096,0.0330
et,Extra Trees Regressor,0.0039,0.0000,0.0048,0.6204,0.0031,0.0075,0.1620
dt,Decision Tree Regressor,0.0049,0.0000,0.0061,0.2471,0.0040,0.0092,0.0310
rf,Random Forest Regressor,0.0044,0.0000,0.0053,0.5618,0.0035,0.0082,0.2070
catboost,CatBoost Regressor,0.0046,0.0000,0.0060,0.5010,0.0039,0.0086,7.0730
ada,AdaBoost Regressor,0.0045,0.0000,0.0056,0.5176,0.0036,0.0085,0.1070
gbr,Gradient Boosting Regressor,0.0042,0.0000,0.0053,0.5605,0.0035,0.0080,0.1400
xgboost,Extreme Gradient Boosting,0.0048,0.0000,0.0060,0.4717,0.0039,0.0091,0.0980
omp,Orthogonal Matching Pursuit,0.0066,0.0001,0.0079,-0.2229,0.0052,0.0124,0.0300
lightgbm,Light Gradient Boosting Machine,0.0055,0.0001,0.0073,0.2358,0.0048,0.0104,0.0590


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [112]:
T01_total_models = []

T01_model_lightgbm = create_model('lightgbm', fold = 5)
T01_total_models.append(T01_model_lightgbm)

T01_model_rf = create_model('rf', fold = 5)
T01_total_models.append(T01_model_rf)

T01_model_gbr = create_model('gbr', fold = 5)
T01_total_models.append(T01_model_gbr)

T01_model_et = create_model('et', fold = 5)
T01_total_models.append(T01_model_et)

T01_model_catboost = create_model('catboost', fold = 5)
T01_total_models.append(T01_model_catboost)

T01_model_xgb = create_model('xgboost', fold = 5)
T01_total_models.append(T01_model_xgb)

T01_model_ada = create_model('ada', fold = 5)
T01_total_models.append(T01_model_ada)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0055,0.0000,0.0067,0.1802,0.0044,0.0104
1,0.0056,0.0001,0.0078,0.3765,0.0051,0.0104
2,0.0041,0.0000,0.0065,-0.2852,0.0043,0.0078
3,0.0067,0.0001,0.0097,0.4681,0.0063,0.0126
4,0.0064,0.0001,0.0087,0.1887,0.0057,0.0118
Mean,0.0057,0.0001,0.0079,0.1857,0.0051,0.0106
Std,0.0009,0.0000,0.0012,0.2600,0.0008,0.0016


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0043,0.0000,0.0054,0.4668,0.0035,0.0080
1,0.0048,0.0000,0.0061,0.6167,0.0040,0.0090
2,0.0041,0.0000,0.0050,0.2375,0.0033,0.0078
3,0.0062,0.0001,0.0079,0.6482,0.0052,0.0117
4,0.0038,0.0000,0.0043,0.7989,0.0028,0.0071
Mean,0.0047,0.0000,0.0058,0.5536,0.0038,0.0087
Std,0.0008,0.0000,0.0012,0.1900,0.0008,0.0016


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0038,0.0000,0.0046,0.6216,0.0030,0.0071
1,0.0049,0.0000,0.0062,0.6044,0.0040,0.0093
2,0.0041,0.0000,0.0057,0.0245,0.0037,0.0077
3,0.0055,0.0001,0.0075,0.6831,0.0049,0.0104
4,0.0043,0.0000,0.0052,0.7154,0.0034,0.0080
Mean,0.0045,0.0000,0.0058,0.5298,0.0038,0.0085
Std,0.0006,0.0000,0.0010,0.2558,0.0007,0.0012


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0035,0.0000,0.0044,0.6421,0.0029,0.0066
1,0.0045,0.0000,0.0060,0.6325,0.0039,0.0085
2,0.0036,0.0000,0.0050,0.2312,0.0033,0.0069
3,0.0057,0.0001,0.0078,0.6583,0.0051,0.0109
4,0.0038,0.0000,0.0042,0.8116,0.0027,0.0070
Mean,0.0042,0.0000,0.0055,0.5951,0.0036,0.0080
Std,0.0008,0.0000,0.0013,0.1933,0.0009,0.0016


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0040,0.0000,0.0050,0.5499,0.0032,0.0075
1,0.0051,0.0000,0.0064,0.5721,0.0042,0.0097
2,0.0033,0.0000,0.0045,0.3895,0.0029,0.0062
3,0.0065,0.0001,0.0082,0.6237,0.0054,0.0122
4,0.0048,0.0000,0.0064,0.5586,0.0042,0.0090
Mean,0.0047,0.0000,0.0061,0.5388,0.0040,0.0089
Std,0.0011,0.0000,0.0013,0.0789,0.0008,0.0021


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [113]:
T01_model_lightgbm = tune_model(T01_model_lightgbm, fold=5, choose_better = True)

T01_model_rf = tune_model(T01_model_rf, fold=5, choose_better = True)

T01_model_gbr = tune_model(T01_model_gbr, fold=5, choose_better = True)

T01_model_et = tune_model(T01_model_et, fold=5, choose_better = True)

T01_model_catboost = tune_model(T01_model_catboost, fold=5, choose_better = True)

T01_model_xgb = tune_model(T01_model_xgb, fold=5, choose_better = True)

T01_model_ada = tune_model(T01_model_ada, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0060,0.0001,0.0075,-0.0286,0.0049,0.0111
1,0.0068,0.0001,0.0098,-0.0033,0.0064,0.0129
2,0.0044,0.0000,0.0061,-0.1257,0.0040,0.0084
3,0.0098,0.0002,0.0134,-0.0089,0.0087,0.0185
4,0.0074,0.0001,0.0100,-0.0725,0.0065,0.0137
Mean,0.0069,0.0001,0.0094,-0.0478,0.0061,0.0129
Std,0.0018,0.0000,0.0025,0.0459,0.0016,0.0033


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0040,0.0000,0.0049,0.5666,0.0032,0.0076
1,0.0055,0.0001,0.0082,0.2990,0.0054,0.0103
2,0.0043,0.0000,0.0060,-0.0753,0.0039,0.0081
3,0.0080,0.0001,0.0112,0.2971,0.0073,0.0150
4,0.0062,0.0001,0.0088,0.1738,0.0057,0.0114
Mean,0.0056,0.0001,0.0078,0.2522,0.0051,0.0105
Std,0.0014,0.0000,0.0022,0.2081,0.0014,0.0027


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0040,0.0000,0.0051,0.5313,0.0033,0.0074
1,0.0048,0.0000,0.0063,0.5957,0.0041,0.0091
2,0.0040,0.0000,0.0051,0.2102,0.0033,0.0076
3,0.0064,0.0001,0.0080,0.6416,0.0052,0.0121
4,0.0045,0.0000,0.0050,0.7304,0.0033,0.0083
Mean,0.0047,0.0000,0.0059,0.5419,0.0038,0.0089
Std,0.0009,0.0000,0.0011,0.1780,0.0008,0.0017


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [114]:
T01_tuned_models = [T01_model_lightgbm, T01_model_rf, T01_model_gbr, T01_model_et, T01_model_catboost, T01_model_xgb, T01_model_ada]

for model in T01_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0043,0.0000,0.0057,0.4056,0.0037,0.0080


,PRODUCT_CODE,X_246,X_247,X_248,X_250,X_251,X_253,X_254,X_256,X_257,...,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,1.247191,-1.0,-1.0,0.000000,0.012146,...,-0.031088,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,-0.333333,0.533273,0.533039
91,0.0,0.0,-0.25,-0.054348,0.0,0.146067,0.0,-1.0,-0.959091,-0.947368,...,-1.072539,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.333333,0.528856,0.523976
92,0.0,1.0,0.25,-0.054348,0.0,0.426966,-1.0,-1.0,-0.281818,-0.356275,...,0.072539,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,0.333333,0.530497,0.526476
93,0.0,0.0,-0.25,1.597826,1.0,0.000000,0.0,0.0,0.609091,0.570850,...,-0.155440,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-1.000000,0.548592,0.545343
94,0.0,0.0,-0.50,-0.054348,0.0,1.876405,-1.0,-1.0,0.713636,0.688259,...,-0.652850,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,0.666667,0.536433,0.537266
95,0.0,0.0,1.00,1.478261,1.0,0.000000,0.0,0.0,0.581818,0.481781,...,0.181347,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.535394,0.533718
96,0.0,0.0,1.25,-0.054348,0.0,1.146067,0.0,-1.0,-0.072727,-0.097166,...,-0.621762,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-0.666667,0.531583,0.534957
97,0.0,0.0,-0.75,-0.054348,0.0,0.123596,-1.0,-1.0,-0.445455,-0.502024,...,-0.238342,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.666667,0.526741,0.522840
98,0.0,0.0,1.00,1.413043,0.0,0.000000,0.0,0.0,0.640909,0.578947,...,0.259067,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,-0.333333,0.538357,0.534345
99,0.0,0.0,1.25,0.663043,0.0,0.000000,0.0,0.0,-0.313636,-0.396761,...,-0.264249,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,-0.333333,0.531043,0.529465


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0036,0.0000,0.0047,0.5962,0.0031,0.0067


,PRODUCT_CODE,X_246,X_247,X_248,X_250,X_251,X_253,X_254,X_256,X_257,...,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,1.247191,-1.0,-1.0,0.000000,0.012146,...,-0.031088,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,-0.333333,0.533273,0.529886
91,0.0,0.0,-0.25,-0.054348,0.0,0.146067,0.0,-1.0,-0.959091,-0.947368,...,-1.072539,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.333333,0.528856,0.525016
92,0.0,1.0,0.25,-0.054348,0.0,0.426966,-1.0,-1.0,-0.281818,-0.356275,...,0.072539,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,0.333333,0.530497,0.528147
93,0.0,0.0,-0.25,1.597826,1.0,0.000000,0.0,0.0,0.609091,0.570850,...,-0.155440,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-1.000000,0.548592,0.542182
94,0.0,0.0,-0.50,-0.054348,0.0,1.876405,-1.0,-1.0,0.713636,0.688259,...,-0.652850,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,0.666667,0.536433,0.529780
95,0.0,0.0,1.00,1.478261,1.0,0.000000,0.0,0.0,0.581818,0.481781,...,0.181347,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.535394,0.534423
96,0.0,0.0,1.25,-0.054348,0.0,1.146067,0.0,-1.0,-0.072727,-0.097166,...,-0.621762,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-0.666667,0.531583,0.529439
97,0.0,0.0,-0.75,-0.054348,0.0,0.123596,-1.0,-1.0,-0.445455,-0.502024,...,-0.238342,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.666667,0.526741,0.530336
98,0.0,0.0,1.00,1.413043,0.0,0.000000,0.0,0.0,0.640909,0.578947,...,0.259067,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,-0.333333,0.538357,0.534452
99,0.0,0.0,1.25,0.663043,0.0,0.000000,0.0,0.0,-0.313636,-0.396761,...,-0.264249,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,-0.333333,0.531043,0.529337


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0032,0.0000,0.0041,0.6897,0.0027,0.0061


...

,PRODUCT_CODE,X_246,X_247,X_248,X_250,X_251,X_253,X_254,X_256,X_257,...,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,1.247191,-1.0,-1.0,0.000000,0.012146,...,-0.031088,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,-0.333333,0.533273,0.533272
91,0.0,0.0,-0.25,-0.054348,0.0,0.146067,0.0,-1.0,-0.959091,-0.947368,...,-1.072539,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.333333,0.528856,0.525747
92,0.0,1.0,0.25,-0.054348,0.0,0.426966,-1.0,-1.0,-0.281818,-0.356275,...,0.072539,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,0.333333,0.530497,0.529464
93,0.0,0.0,-0.25,1.597826,1.0,0.000000,0.0,0.0,0.609091,0.570850,...,-0.155440,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-1.000000,0.548592,0.546242
94,0.0,0.0,-0.50,-0.054348,0.0,1.876405,-1.0,-1.0,0.713636,0.688259,...,-0.652850,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,0.666667,0.536433,0.536137
95,0.0,0.0,1.00,1.478261,1.0,0.000000,0.0,0.0,0.581818,0.481781,...,0.181347,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.535394,0.538986
96,0.0,0.0,1.25,-0.054348,0.0,1.146067,0.0,-1.0,-0.072727,-0.097166,...,-0.621762,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-0.666667,0.531583,0.532051
97,0.0,0.0,-0.75,-0.054348,0.0,0.123596,-1.0,-1.0,-0.445455,-0.502024,...,-0.238342,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.666667,0.526741,0.527721
98,0.0,0.0,1.00,1.413043,0.0,0.000000,0.0,0.0,0.640909,0.578947,...,0.259067,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,-0.333333,0.538357,0.534114
99,0.0,0.0,1.25,0.663043,0.0,0.000000,0.0,0.0,-0.313636,-0.396761,...,-0.264249,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,-0.333333,0.531043,0.531674


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.0039,0.0000,0.0050,0.5485,0.0032,0.0072


,PRODUCT_CODE,X_246,X_247,X_248,X_250,X_251,X_253,X_254,X_256,X_257,...,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,1.247191,-1.0,-1.0,0.000000,0.012146,...,-0.031088,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,-0.333333,0.533273,0.535379
91,0.0,0.0,-0.25,-0.054348,0.0,0.146067,0.0,-1.0,-0.959091,-0.947368,...,-1.072539,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.333333,0.528856,0.528361
92,0.0,1.0,0.25,-0.054348,0.0,0.426966,-1.0,-1.0,-0.281818,-0.356275,...,0.072539,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,0.333333,0.530497,0.529090
93,0.0,0.0,-0.25,1.597826,1.0,0.000000,0.0,0.0,0.609091,0.570850,...,-0.155440,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-1.000000,0.548592,0.543164
94,0.0,0.0,-0.50,-0.054348,0.0,1.876405,-1.0,-1.0,0.713636,0.688259,...,-0.652850,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,0.666667,0.536433,0.529829
95,0.0,0.0,1.00,1.478261,1.0,0.000000,0.0,0.0,0.581818,0.481781,...,0.181347,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.535394,0.536336
96,0.0,0.0,1.25,-0.054348,0.0,1.146067,0.0,-1.0,-0.072727,-0.097166,...,-0.621762,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-0.666667,0.531583,0.528174
97,0.0,0.0,-0.75,-0.054348,0.0,0.123596,-1.0,-1.0,-0.445455,-0.502024,...,-0.238342,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.666667,0.526741,0.527485
98,0.0,0.0,1.00,1.413043,0.0,0.000000,0.0,0.0,0.640909,0.578947,...,0.259067,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,-0.333333,0.538357,0.531194
99,0.0,0.0,1.25,0.663043,0.0,0.000000,0.0,0.0,-0.313636,-0.396761,...,-0.264249,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,-0.333333,0.531043,0.529122


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0040,0.0000,0.0052,0.5124,0.0034,0.0074


,PRODUCT_CODE,X_246,X_247,X_248,X_250,X_251,X_253,X_254,X_256,X_257,...,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,1.247191,-1.0,-1.0,0.000000,0.012146,...,-0.031088,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,-0.333333,0.533273,0.529603
91,0.0,0.0,-0.25,-0.054348,0.0,0.146067,0.0,-1.0,-0.959091,-0.947368,...,-1.072539,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.333333,0.528856,0.528123
92,0.0,1.0,0.25,-0.054348,0.0,0.426966,-1.0,-1.0,-0.281818,-0.356275,...,0.072539,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,0.333333,0.530497,0.529620
93,0.0,0.0,-0.25,1.597826,1.0,0.000000,0.0,0.0,0.609091,0.570850,...,-0.155440,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-1.000000,0.548592,0.538816
94,0.0,0.0,-0.50,-0.054348,0.0,1.876405,-1.0,-1.0,0.713636,0.688259,...,-0.652850,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,0.666667,0.536433,0.530315
95,0.0,0.0,1.00,1.478261,1.0,0.000000,0.0,0.0,0.581818,0.481781,...,0.181347,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.535394,0.535458
96,0.0,0.0,1.25,-0.054348,0.0,1.146067,0.0,-1.0,-0.072727,-0.097166,...,-0.621762,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-0.666667,0.531583,0.529603
97,0.0,0.0,-0.75,-0.054348,0.0,0.123596,-1.0,-1.0,-0.445455,-0.502024,...,-0.238342,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.666667,0.526741,0.529603
98,0.0,0.0,1.00,1.413043,0.0,0.000000,0.0,0.0,0.640909,0.578947,...,0.259067,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,-0.333333,0.538357,0.535458
99,0.0,0.0,1.25,0.663043,0.0,0.000000,0.0,0.0,-0.313636,-0.396761,...,-0.264249,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,-0.333333,0.531043,0.529614


In [115]:
T01_final_regression_model = finalize_model(T01_model_gbr)

T01_prediction = predict_model(T01_final_regression_model, data=T01_data)

In [116]:
T01_datas = T01_prediction['prediction_label']
T01_datas = pd.DataFrame(T01_datas.values, columns = ['Y_Quality'])
T01_datas

,Y_Quality
0,0.521619
1,0.526817
2,0.531680
3,0.530477
4,0.528963
...,...
124,0.534920
125,0.533509
126,0.534384
127,0.560159


In [117]:
T01_xtrain, T01_xtest, T01_ytrain, T01_ytest = train_test_split(T01_datas, T01_class, test_size = 0.1, random_state = 4)

In [118]:
scores = []
for thisC in [*range(1,100)]:
    svc=SVC(kernel='rbf',C=thisC, decision_function_shape = 'ovo')
    model=svc.fit(T01_xtrain,T01_ytrain)
    scoreTrain=model.score(T01_xtrain,T01_ytrain)
    scoreTest=model.score(T01_xtest,T01_ytest)
    print("선형 SVM : C:{}, training score:{:2f}, test score:{:2f}".format
          (thisC,scoreTrain, scoreTest))
    scores.append([scoreTrain, scoreTest])

선형 SVM : C:1, training score:0.965517, test score:0.923077
선형 SVM : C:2, training score:0.991379, test score:1.000000
선형 SVM : C:3, training score:0.982759, test score:1.000000
선형 SVM : C:4, training score:0.974138, test score:1.000000
선형 SVM : C:5, training score:0.982759, test score:1.000000
선형 SVM : C:6, training score:0.982759, test score:1.000000
선형 SVM : C:7, training score:0.982759, test score:1.000000
선형 SVM : C:8, training score:0.982759, test score:1.000000
선형 SVM : C:9, training score:0.982759, test score:1.000000
선형 SVM : C:10, training score:0.982759, test score:1.000000
선형 SVM : C:11, training score:0.982759, test score:1.000000
선형 SVM : C:12, training score:0.982759, test score:1.000000
선형 SVM : C:13, training score:0.982759, test score:1.000000
선형 SVM : C:14, training score:0.982759, test score:1.000000
선형 SVM : C:15, training score:0.982759, test score:1.000000
선형 SVM : C:16, training score:0.982759, test score:1.000000
선형 SVM : C:17, training score:0.982759, test scor

In [119]:
T01_svm = svm.SVC(kernel = 'rbf', decision_function_shape = 'ovo', C = 2)
T01_svm.fit(T01_xtrain, T01_ytrain)
T01_svm.predict(T01_xtest)

array([1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2], dtype=int64)

# T10

In [120]:
T10_class = T10['Y_Class']
T10_target = T10['Y_Quality']
T10_data = T10.drop(['Y_Class', 'Y_Quality'], axis = 1)

In [121]:
T_10c = T10_data.columns
T_10scaler = RobustScaler()
T_10scaled = T_10scaler.fit_transform(T10_data)
T10_data = pd.DataFrame(T_10scaled, columns = T_10c)

T10_y_class = pd.DataFrame(T10_target, columns = ['Y_Quality'])
T10_idx = T10_y_class.index
T10_y_class = T10_y_class.reset_index()
T10_data = pd.concat([T10_data, T10_y_class], axis = 1)
T10_data = T10_data.set_index('index')
T10_data.drop(['Y_Quality'], axis = 1, inplace = True)
T10_data

,PRODUCT_CODE,X_1,X_2,X_5,X_7,X_8,X_9,X_11,X_12,X_13,...,X_925,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,DAY_OF_WEEK
index,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,1.4,1.0,0.000000,0.0,0.0,0.653153,0.746725,0.714286,...,0.586957,0.962572,0.66,0.0,0.386364,0.111111,1.0,0.584058,0.0,-1.000000
23,0.0,0.0,1.4,1.0,0.000000,0.0,0.0,0.662162,0.768559,0.723214,...,-34.804348,-45.815738,-29.52,-32.6,-38.275000,-32.111111,-136.0,-180.608692,-13.2,-1.000000
25,0.0,0.0,0.4,1.0,0.000000,0.0,0.0,0.788288,0.917031,0.852679,...,-0.652174,0.022073,0.66,0.1,0.581818,1.000000,1.0,0.739130,0.0,0.666667
29,0.0,0.0,1.0,1.0,0.000000,0.0,0.0,0.747748,0.917031,0.790179,...,0.260870,0.606526,0.30,0.1,0.738636,1.000000,1.0,0.304348,0.0,0.666667
37,0.0,0.0,1.0,1.0,0.000000,0.0,0.0,0.792793,0.947598,0.830357,...,0.608696,0.544146,0.16,0.1,0.756818,1.000000,1.0,0.763768,0.1,-0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,0.0,-1.0,-0.2,0.0,0.833333,0.0,1.0,-1.549550,-1.519651,-1.504464,...,-0.043478,0.092131,0.30,0.0,0.634091,1.000000,1.0,1.130435,0.1,-1.000000
590,0.0,-1.0,-1.2,0.0,0.833333,0.0,1.0,-1.527027,-1.506550,-1.419643,...,-0.434783,0.241843,0.48,0.1,0.900000,1.111111,0.0,-0.086956,0.0,-0.666667
591,0.0,-1.0,-1.6,0.0,0.833333,0.0,1.0,-1.531532,-1.506550,-1.473214,...,-0.108696,0.231286,0.02,0.0,0.909091,1.000000,0.0,0.000000,0.0,-0.666667


In [122]:
#13
from pycaret.regression import *
setup_rgs = setup(data = T10_data, target = T10_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,4
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(349, 577)"
4,Transformed data shape,"(349, 577)"
5,Transformed train set shape,"(244, 577)"
6,Transformed test set shape,"(105, 577)"
7,Numeric features,576
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,0.0030,0.0000,0.0045,-0.0502,0.0029,0.0057,0.0270
catboost,CatBoost Regressor,0.0029,0.0000,0.0042,0.0747,0.0028,0.0055,7.5430
en,Elastic Net,0.0030,0.0000,0.0045,-0.0502,0.0029,0.0057,0.0270
xgboost,Extreme Gradient Boosting,0.0031,0.0000,0.0043,-0.1034,0.0028,0.0058,0.1180
llar,Lasso Least Angle Regression,0.0030,0.0000,0.0045,-0.0502,0.0029,0.0057,0.0250
br,Bayesian Ridge,0.0035,0.0000,0.0047,-0.2289,0.0031,0.0066,0.0280
knn,K Neighbors Regressor,0.0034,0.0000,0.0047,-0.1694,0.0030,0.0065,0.0270
dt,Decision Tree Regressor,0.0041,0.0000,0.0058,-1.0183,0.0038,0.0077,0.0320
rf,Random Forest Regressor,0.0032,0.0000,0.0045,-0.0894,0.0029,0.0060,0.4020
et,Extra Trees Regressor,0.0031,0.0000,0.0042,-0.0471,0.0028,0.0058,0.2820


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [123]:
T10_total_models = []

T10_model_rf = create_model('rf', fold = 5)
T10_total_models.append(T10_model_rf)

T10_model_gbr = create_model('gbr', fold = 5)
T10_total_models.append(T10_model_gbr)

T10_model_et = create_model('et', fold = 5)
T10_total_models.append(T10_model_et)

T10_model_catboost = create_model('catboost', fold = 5)
T10_total_models.append(T10_model_catboost)

T10_model_xgb = create_model('xgboost', fold = 5)
T10_total_models.append(T10_model_xgb)

T10_model_ada = create_model('ada', fold = 5)
T10_total_models.append(T10_model_ada)

T10_model_lightgbm = create_model('lightgbm', fold = 5)
T10_total_models.append(T10_model_lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0032,0.0000,0.0046,0.0544,0.0030,0.0061
1,0.0033,0.0000,0.0051,0.0132,0.0033,0.0063
2,0.0029,0.0000,0.0042,-0.1798,0.0028,0.0054
3,0.0029,0.0000,0.0038,-0.1422,0.0025,0.0054
4,0.0037,0.0000,0.0051,0.0748,0.0034,0.0070
Mean,0.0032,0.0000,0.0046,-0.0359,0.0030,0.0060
Std,0.0003,0.0000,0.0005,0.1047,0.0003,0.0006


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0030,0.0000,0.0041,0.2754,0.0027,0.0057
1,0.0031,0.0000,0.0045,0.2206,0.0030,0.0058
2,0.0030,0.0000,0.0043,-0.2221,0.0028,0.0056
3,0.0029,0.0000,0.0038,-0.1195,0.0025,0.0054
4,0.0035,0.0000,0.0052,0.0425,0.0034,0.0067
Mean,0.0031,0.0000,0.0044,0.0394,0.0029,0.0058
Std,0.0002,0.0000,0.0005,0.1909,0.0003,0.0004


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0030,0.0000,0.0043,0.1787,0.0028,0.0057
1,0.0033,0.0000,0.0048,0.1142,0.0031,0.0063
2,0.0028,0.0000,0.0039,0.0228,0.0025,0.0052
3,0.0026,0.0000,0.0032,0.1838,0.0021,0.0049
4,0.0037,0.0000,0.0050,0.1203,0.0033,0.0070
Mean,0.0031,0.0000,0.0042,0.1240,0.0028,0.0058
Std,0.0004,0.0000,0.0006,0.0582,0.0004,0.0007


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0030,0.0000,0.0046,0.0777,0.0030,0.0057
1,0.0034,0.0000,0.0051,-0.0094,0.0034,0.0064
2,0.0028,0.0000,0.0039,0.0008,0.0025,0.0053
3,0.0026,0.0000,0.0039,-0.1589,0.0025,0.0048
4,0.0037,0.0000,0.0056,-0.0968,0.0037,0.0071
Mean,0.0031,0.0000,0.0046,-0.0373,0.0030,0.0059
Std,0.0004,0.0000,0.0007,0.0822,0.0004,0.0008


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0035,0.0000,0.0050,-0.1032,0.0033,0.0067
1,0.0036,0.0000,0.0052,-0.0492,0.0034,0.0068
2,0.0031,0.0000,0.0043,-0.1882,0.0028,0.0058
3,0.0029,0.0000,0.0040,-0.2357,0.0026,0.0055
4,0.0042,0.0000,0.0058,-0.1956,0.0038,0.0079
Mean,0.0035,0.0000,0.0049,-0.1544,0.0032,0.0065
Std,0.0004,0.0000,0.0007,0.0680,0.0004,0.0009


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [124]:
T10_model_rf = tune_model(T10_model_rf, fold=5, choose_better = True)

T10_model_gbr = tune_model(T10_model_gbr, fold=5, choose_better = True)

T10_model_et = tune_model(T10_model_et, fold=5, choose_better = True)

T10_model_catboost = tune_model(T10_model_catboost, fold=5, choose_better = True)

T10_model_xgb = tune_model(T10_model_xgb, fold=5, choose_better = True)

T10_model_ada = tune_model(T10_model_ada, fold=5, choose_better = True)

T10_model_lightgbm = tune_model(T10_model_lightgbm, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0031,0.0000,0.0047,0.0344,0.0031,0.0058
1,0.0032,0.0000,0.0051,-0.0007,0.0034,0.0061
2,0.0027,0.0000,0.0038,0.0592,0.0025,0.0052
3,0.0028,0.0000,0.0039,-0.1946,0.0026,0.0053
4,0.0035,0.0000,0.0053,0.0137,0.0035,0.0067
Mean,0.0031,0.0000,0.0046,-0.0176,0.0030,0.0058
Std,0.0003,0.0000,0.0006,0.0908,0.0004,0.0006


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0031,0.0000,0.0046,0.0876,0.0030,0.0059
1,0.0032,0.0000,0.0049,0.0804,0.0032,0.0061
2,0.0030,0.0000,0.0040,-0.0636,0.0026,0.0056
3,0.0029,0.0000,0.0040,-0.2293,0.0026,0.0055
4,0.0035,0.0000,0.0052,0.0326,0.0034,0.0066
Mean,0.0032,0.0000,0.0045,-0.0184,0.0030,0.0060
Std,0.0002,0.0000,0.0005,0.1184,0.0003,0.0004


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0031,0.0000,0.0048,-0.0000,0.0031,0.0058
1,0.0032,0.0000,0.0051,-0.0121,0.0034,0.0062
2,0.0028,0.0000,0.0039,-0.0000,0.0025,0.0054
3,0.0026,0.0000,0.0038,-0.0903,0.0024,0.0048
4,0.0034,0.0000,0.0054,-0.0105,0.0035,0.0065
Mean,0.0030,0.0000,0.0046,-0.0226,0.0030,0.0057
Std,0.0003,0.0000,0.0006,0.0342,0.0004,0.0006


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [125]:
T10_tuned_models = [T10_model_rf, T10_model_gbr, T10_model_et, T10_model_catboost, T10_model_xgb, T10_model_ada, T10_model_lightgbm]

for model in T10_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0034,0.0000,0.0049,0.0227,0.0032,0.0064


,PRODUCT_CODE,X_1,X_2,X_5,X_7,X_8,X_9,X_11,X_12,X_13,...,X_927,X_928,X_929,X_930,X_931,X_932,X_933,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.151786,...,0.54,0.0,0.531818,0.111111,1.0,0.434783,0.1,0.666667,0.525525,0.530737
245,0.0,0.0,0.4,1.0,0.000000,0.0,0.0,-0.373874,-0.336245,-0.241071,...,0.12,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.666667,0.531676,0.530481
246,0.0,0.0,-1.0,0.0,0.833333,0.0,1.0,-0.707207,-0.698690,-0.633929,...,-0.52,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.333333,0.529952,0.530713
247,0.0,-1.0,-0.2,0.0,0.833333,0.0,1.0,-1.788288,-1.733624,-1.723214,...,0.86,0.1,0.972727,1.000000,1.0,0.449275,0.0,-1.000000,0.531376,0.530643
248,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.716216,0.724891,0.843750,...,-0.24,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,0.666667,0.528473,0.530233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,1.333333,0.0,1.0,-0.225225,-0.139738,-0.272321,...,-0.50,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.333333,0.526997,0.530566
345,0.0,0.0,-1.2,0.0,2.000000,0.0,1.0,-2.954955,-2.799127,-2.910714,...,-1.32,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.333333,0.537283,0.530011
346,0.0,0.0,-0.2,0.0,1.500000,0.0,1.0,-0.639640,-0.589520,-0.580357,...,-0.18,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,0.000000,0.527637,0.530556
347,0.0,-1.0,1.4,0.0,0.000000,0.0,0.0,0.932432,1.043668,0.964286,...,0.86,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.333333,0.525897,0.530678


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0030,0.0000,0.0038,0.4100,0.0025,0.0057


,PRODUCT_CODE,X_1,X_2,X_5,X_7,X_8,X_9,X_11,X_12,X_13,...,X_927,X_928,X_929,X_930,X_931,X_932,X_933,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.151786,...,0.54,0.0,0.531818,0.111111,1.0,0.434783,0.1,0.666667,0.525525,0.530220
245,0.0,0.0,0.4,1.0,0.000000,0.0,0.0,-0.373874,-0.336245,-0.241071,...,0.12,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.666667,0.531676,0.530633
246,0.0,0.0,-1.0,0.0,0.833333,0.0,1.0,-0.707207,-0.698690,-0.633929,...,-0.52,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.333333,0.529952,0.528516
247,0.0,-1.0,-0.2,0.0,0.833333,0.0,1.0,-1.788288,-1.733624,-1.723214,...,0.86,0.1,0.972727,1.000000,1.0,0.449275,0.0,-1.000000,0.531376,0.532164
248,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.716216,0.724891,0.843750,...,-0.24,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,0.666667,0.528473,0.530404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,1.333333,0.0,1.0,-0.225225,-0.139738,-0.272321,...,-0.50,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.333333,0.526997,0.531342
345,0.0,0.0,-1.2,0.0,2.000000,0.0,1.0,-2.954955,-2.799127,-2.910714,...,-1.32,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.333333,0.537283,0.534180
346,0.0,0.0,-0.2,0.0,1.500000,0.0,1.0,-0.639640,-0.589520,-0.580357,...,-0.18,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,0.000000,0.527637,0.530382
347,0.0,-1.0,1.4,0.0,0.000000,0.0,0.0,0.932432,1.043668,0.964286,...,0.86,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.333333,0.525897,0.530041


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.0029,0.0000,0.0038,0.4115,0.0025,0.0055


...

,PRODUCT_CODE,X_1,X_2,X_5,X_7,X_8,X_9,X_11,X_12,X_13,...,X_927,X_928,X_929,X_930,X_931,X_932,X_933,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.151786,...,0.54,0.0,0.531818,0.111111,1.0,0.434783,0.1,0.666667,0.525525,0.529878
245,0.0,0.0,0.4,1.0,0.000000,0.0,0.0,-0.373874,-0.336245,-0.241071,...,0.12,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.666667,0.531676,0.531130
246,0.0,0.0,-1.0,0.0,0.833333,0.0,1.0,-0.707207,-0.698690,-0.633929,...,-0.52,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.333333,0.529952,0.527844
247,0.0,-1.0,-0.2,0.0,0.833333,0.0,1.0,-1.788288,-1.733624,-1.723214,...,0.86,0.1,0.972727,1.000000,1.0,0.449275,0.0,-1.000000,0.531376,0.531153
248,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.716216,0.724891,0.843750,...,-0.24,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,0.666667,0.528473,0.527475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,1.333333,0.0,1.0,-0.225225,-0.139738,-0.272321,...,-0.50,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.333333,0.526997,0.532280
345,0.0,0.0,-1.2,0.0,2.000000,0.0,1.0,-2.954955,-2.799127,-2.910714,...,-1.32,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.333333,0.537283,0.535911
346,0.0,0.0,-0.2,0.0,1.500000,0.0,1.0,-0.639640,-0.589520,-0.580357,...,-0.18,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,0.000000,0.527637,0.529630
347,0.0,-1.0,1.4,0.0,0.000000,0.0,0.0,0.932432,1.043668,0.964286,...,0.86,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.333333,0.525897,0.527339


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0032,0.0000,0.0046,0.1357,0.0030,0.0061


,PRODUCT_CODE,X_1,X_2,X_5,X_7,X_8,X_9,X_11,X_12,X_13,...,X_927,X_928,X_929,X_930,X_931,X_932,X_933,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.151786,...,0.54,0.0,0.531818,0.111111,1.0,0.434783,0.1,0.666667,0.525525,0.530344
245,0.0,0.0,0.4,1.0,0.000000,0.0,0.0,-0.373874,-0.336245,-0.241071,...,0.12,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.666667,0.531676,0.530418
246,0.0,0.0,-1.0,0.0,0.833333,0.0,1.0,-0.707207,-0.698690,-0.633929,...,-0.52,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.333333,0.529952,0.530344
247,0.0,-1.0,-0.2,0.0,0.833333,0.0,1.0,-1.788288,-1.733624,-1.723214,...,0.86,0.1,0.972727,1.000000,1.0,0.449275,0.0,-1.000000,0.531376,0.530502
248,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.716216,0.724891,0.843750,...,-0.24,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,0.666667,0.528473,0.530294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,1.333333,0.0,1.0,-0.225225,-0.139738,-0.272321,...,-0.50,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.333333,0.526997,0.530383
345,0.0,0.0,-1.2,0.0,2.000000,0.0,1.0,-2.954955,-2.799127,-2.910714,...,-1.32,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.333333,0.537283,0.530599
346,0.0,0.0,-0.2,0.0,1.500000,0.0,1.0,-0.639640,-0.589520,-0.580357,...,-0.18,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,0.000000,0.527637,0.530424
347,0.0,-1.0,1.4,0.0,0.000000,0.0,0.0,0.932432,1.043668,0.964286,...,0.86,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.333333,0.525897,0.530502


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0034,0.0000,0.0050,-0.0000,0.0033,0.0063


,PRODUCT_CODE,X_1,X_2,X_5,X_7,X_8,X_9,X_11,X_12,X_13,...,X_927,X_928,X_929,X_930,X_931,X_932,X_933,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.151786,...,0.54,0.0,0.531818,0.111111,1.0,0.434783,0.1,0.666667,0.525525,0.530323
245,0.0,0.0,0.4,1.0,0.000000,0.0,0.0,-0.373874,-0.336245,-0.241071,...,0.12,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.666667,0.531676,0.530323
246,0.0,0.0,-1.0,0.0,0.833333,0.0,1.0,-0.707207,-0.698690,-0.633929,...,-0.52,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.333333,0.529952,0.530323
247,0.0,-1.0,-0.2,0.0,0.833333,0.0,1.0,-1.788288,-1.733624,-1.723214,...,0.86,0.1,0.972727,1.000000,1.0,0.449275,0.0,-1.000000,0.531376,0.530323
248,0.0,0.0,-0.4,1.0,0.000000,0.0,0.0,0.716216,0.724891,0.843750,...,-0.24,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,0.666667,0.528473,0.530323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,1.333333,0.0,1.0,-0.225225,-0.139738,-0.272321,...,-0.50,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.333333,0.526997,0.530323
345,0.0,0.0,-1.2,0.0,2.000000,0.0,1.0,-2.954955,-2.799127,-2.910714,...,-1.32,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.333333,0.537283,0.530323
346,0.0,0.0,-0.2,0.0,1.500000,0.0,1.0,-0.639640,-0.589520,-0.580357,...,-0.18,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,0.000000,0.527637,0.530323
347,0.0,-1.0,1.4,0.0,0.000000,0.0,0.0,0.932432,1.043668,0.964286,...,0.86,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.333333,0.525897,0.530323


In [126]:
T10_final_regression_model = finalize_model(T10_model_et)

T10_prediction = predict_model(T10_final_regression_model, data=T10_data)

In [127]:
T10_datas = T10_prediction['prediction_label']
T10_datas = pd.DataFrame(T10_datas.values, columns = ['Y_Quality'])
T10_datas

,Y_Quality
0,0.517719
1,0.519090
2,0.529362
3,0.532405
4,0.533110
...,...
344,0.529510
345,0.529948
346,0.529308
347,0.526546


In [128]:
T10_xtrain, T10_xtest, T10_ytrain, T10_ytest = train_test_split(T10_datas, T10_class, test_size = 0.1, random_state = 4)

In [129]:
scores = []
for thisC in [*range(1,100)]:
    svc=SVC(kernel='rbf',C=thisC, decision_function_shape = 'ovo')
    model=svc.fit(T10_xtrain,T10_ytrain)
    scoreTrain=model.score(T10_xtrain,T10_ytrain)
    scoreTest=model.score(T10_xtest,T10_ytest)
    print("선형 SVM : C:{}, training score:{:2f}, test score:{:2f}".format
          (thisC,scoreTrain, scoreTest))
    scores.append([scoreTrain, scoreTest])

선형 SVM : C:1, training score:0.990446, test score:1.000000
선형 SVM : C:2, training score:0.990446, test score:1.000000
선형 SVM : C:3, training score:0.990446, test score:1.000000
선형 SVM : C:4, training score:0.990446, test score:1.000000
선형 SVM : C:5, training score:0.990446, test score:1.000000
선형 SVM : C:6, training score:0.990446, test score:1.000000
선형 SVM : C:7, training score:0.990446, test score:1.000000
선형 SVM : C:8, training score:0.990446, test score:1.000000
선형 SVM : C:9, training score:0.990446, test score:1.000000
선형 SVM : C:10, training score:0.993631, test score:1.000000
선형 SVM : C:11, training score:0.993631, test score:1.000000
선형 SVM : C:12, training score:1.000000, test score:1.000000
선형 SVM : C:13, training score:1.000000, test score:1.000000
선형 SVM : C:14, training score:1.000000, test score:1.000000
선형 SVM : C:15, training score:1.000000, test score:1.000000
선형 SVM : C:16, training score:1.000000, test score:1.000000
선형 SVM : C:17, training score:1.000000, test scor

In [130]:
T10_svm = svm.SVC(kernel = 'rbf', decision_function_shape = 'ovo', C = 12)
T10_svm.fit(T10_xtrain, T10_ytrain)
T10_svm.predict(T10_xtest)

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1], dtype=int64)

# Test

In [131]:
test_data = pd.read_csv("./test.csv")
test_data

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,TEST_305,2022-11-05 11:18,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,TEST_306,2022-11-05 16:39,T100304,T_31,2.0,96.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,TEST_307,2022-11-05 16:47,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,TEST_308,2022-11-05 20:53,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
test_data['YEAR'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.year
test_data['MONTH'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.month
test_data['DAY'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.day
test_data['HOUR'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.hour
test_data['MINUTE'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.minute
test_data['DAY_OF_WEEK'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.day_name()

test_data = test_data.drop(['MONTH', 'DAY', 'YEAR', 'MINUTE', 'HOUR'], axis = 1)

In [133]:
test_data.drop(colls, inplace = True, axis = 1)

In [134]:
test_data.loc[test_data.DAY_OF_WEEK == 'Monday','DAY_OF_WEEK'] = 1
test_data.loc[test_data.DAY_OF_WEEK == 'Tuesday','DAY_OF_WEEK'] = 2
test_data.loc[test_data.DAY_OF_WEEK == 'Wednesday','DAY_OF_WEEK'] = 3
test_data.loc[test_data.DAY_OF_WEEK == 'Thursday','DAY_OF_WEEK'] = 4
test_data.loc[test_data.DAY_OF_WEEK == 'Friday','DAY_OF_WEEK'] = 5
test_data.loc[test_data.DAY_OF_WEEK == 'Saturday','DAY_OF_WEEK'] = 6
test_data.loc[test_data.DAY_OF_WEEK == 'Sunday','DAY_OF_WEEK'] = 7

test_data.loc[test_data.PRODUCT_CODE == 'A_31','PRODUCT_CODE'] = 1
test_data.loc[test_data.PRODUCT_CODE == 'T_31','PRODUCT_CODE'] = 2
test_data.loc[test_data.PRODUCT_CODE == 'O_31','PRODUCT_CODE'] = 3

test_data.drop(['PRODUCT_ID', 'TIMESTAMP'], axis = 1, inplace = True)
test_data

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,DAY_OF_WEEK
0,T100306,2,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
1,T100304,2,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
2,T100304,2,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3,T010305,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,467.0,444.192308,423.0,NaN,NaN,NaN,NaN,NaN,NaN,5
4,T010306,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,465.0,443.211539,423.0,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,T100306,2,2.0,91.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
306,T100304,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
307,T100306,2,2.0,91.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
308,T100306,2,2.0,95.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6


In [135]:
TT050304 = test_data[test_data['LINE'] == 'T050304']
TT050307 = test_data[test_data['LINE'] == 'T050307']
TT010305 = test_data[test_data['LINE'] == 'T010305']
TT010306 = test_data[test_data['LINE'] == 'T010306']
TT100304 = test_data[test_data['LINE'] == 'T100304']
TT100306 = test_data[test_data['LINE'] == 'T100306']

TT050304.drop(['LINE'], axis = 1, inplace = True)
TT050307.drop(['LINE'], axis = 1, inplace = True)
TT010305.drop(['LINE'], axis = 1, inplace = True)
TT010306.drop(['LINE'], axis = 1, inplace = True)
TT100304.drop(['LINE'], axis = 1, inplace = True)
TT100306.drop(['LINE'], axis = 1, inplace = True)

In [136]:
TT050304.drop(T050304_col, inplace = True, axis = 1)
TT050307.drop(T050307_col, inplace = True, axis = 1)
TT010305.drop(T010305_col, inplace = True, axis = 1)
TT010306.drop(T010306_col, inplace = True, axis = 1)
TT100304.drop(T100304_col, inplace = True, axis = 1)
TT100306.drop(T100306_col, inplace = True, axis = 1)

In [137]:
TT05 = pd.concat([TT050304, TT050307])
TT01 = pd.concat([TT010305, TT010306])
TT10 = pd.concat([TT100304, TT100306])

In [138]:
TT05.drop(T5drop, inplace = True, axis = 1)
TT01.drop(T0drop, inplace = True, axis = 1)
TT10.drop(T1drop, inplace = True, axis = 1)

In [139]:
#TT05 = TT05.fillna(T05.mean())
#TT01 = TT01.fillna(T01.mean())
#TT10 = TT10.fillna(T10.mean())

TT05 = TT05.fillna(0)
TT01 = TT01.fillna(0)
TT10 = TT10.fillna(0)

In [140]:
TT05.drop(T05_cols, inplace = True, axis = 1)
TT01.drop(T01_cols, inplace = True, axis = 1)
TT10.drop(T10_cols, inplace = True, axis = 1)

In [141]:
TT05_idx = TT05['PRODUCT_CODE'].index
TT05_idx = pd.DataFrame(TT05_idx, columns = ['index'])

TT05c = TT05.columns
TT05scaled = T_05scaler.transform(TT05)
TT05scaled
TT05 = pd.DataFrame(TT05scaled, columns = TT05c)

TT05 = pd.concat([TT05_idx, TT05], axis = 1)
TT05 = TT05.set_index('index')

In [142]:
TT01_idx = TT01['PRODUCT_CODE'].index
TT01_idx = pd.DataFrame(TT01_idx, columns = ['index'])

TT01c = TT01.columns
TT01scaled = T_01scaler.transform(TT01)
TT01 = pd.DataFrame(TT01scaled, columns = TT01c)

TT01 = pd.concat([TT01_idx, TT01], axis = 1)
TT01 = TT01.set_index('index')

In [143]:
TT10_idx = TT10['PRODUCT_CODE'].index
TT10_idx = pd.DataFrame(TT10_idx, columns = ['index'])

TT10c = TT10.columns
TT10scaled = T_10scaler.transform(TT10)
TT10 = pd.DataFrame(TT10scaled, columns = TT10c)

TT10 = pd.concat([TT10_idx, TT10], axis = 1)
TT10 = TT10.set_index('index')

In [144]:
from pycaret.regression import *
TT05_prediction = predict_model(T05_final_regression_model, data=TT05)
TT01_prediction = predict_model(T01_final_regression_model, data=TT01)
TT10_prediction = predict_model(T10_final_regression_model, data=TT10)

TT05_prediction = pd.DataFrame(TT05_prediction, columns = ['prediction_label'])
TT01_prediction = pd.DataFrame(TT01_prediction, columns = ['prediction_label'])
TT10_prediction = pd.DataFrame(TT10_prediction, columns = ['prediction_label'])

In [145]:
TT05_y_class = pd.DataFrame(TT05, columns = ['PRODUCT_CODE'])
TT05_idx = TT05_y_class.index
TT05_y_class = TT05_y_class.reset_index()
TT05_prediction = pd.concat([TT05_prediction, TT05_y_class], axis = 1)
TT05_prediction = TT05_prediction.set_index('index')
TT05_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

TT01_y_class = pd.DataFrame(TT01, columns = ['PRODUCT_CODE'])
TT01_idx = TT01_y_class.index
TT01_y_class = TT01_y_class.reset_index()
TT01_prediction = pd.concat([TT01_prediction, TT01_y_class], axis = 1)
TT01_prediction = TT01_prediction.set_index('index')
TT01_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

TT10_y_class = pd.DataFrame(TT10, columns = ['PRODUCT_CODE'])
TT10_idx = TT10_y_class.index
TT10_y_class = TT10_y_class.reset_index()
TT10_prediction = pd.concat([TT10_prediction, TT10_y_class], axis = 1)
TT10_prediction = TT10_prediction.set_index('index')
TT10_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

In [146]:
TT05_datas = TT05_prediction['prediction_label']
TT05_datas = pd.DataFrame(TT05_datas.values, columns = ['Y_Quality'])

TT05y_idx= TT05_prediction.index
TT05y_idx = pd.DataFrame(TT05y_idx, columns = ['index'])
TT05_datas = pd.concat([TT05y_idx, TT05_datas], axis = 1)
TT05_datas = TT05_datas.set_index('index')

TT01_datas = TT01_prediction['prediction_label']
TT01_datas = pd.DataFrame(TT01_datas.values, columns = ['Y_Quality'])

TT01y_idx= TT01_prediction.index
TT01y_idx = pd.DataFrame(TT01y_idx, columns = ['index'])
TT01_datas = pd.concat([TT01y_idx, TT01_datas], axis = 1)
TT01_datas = TT01_datas.set_index('index')

TT10_datas = TT10_prediction['prediction_label']
TT10_datas = pd.DataFrame(TT10_datas.values, columns = ['Y_Quality'])

TT10y_idx= TT10_prediction.index
TT10y_idx = pd.DataFrame(TT10y_idx, columns = ['index'])
TT10_datas = pd.concat([TT10y_idx, TT10_datas], axis = 1)
TT10_datas = TT10_datas.set_index('index')

In [147]:
TT05_predictions = T05_svm.predict(TT05_datas)
TT01_predictions = T01_svm.predict(TT01_datas)
TT10_predictions = T10_svm.predict(TT10_datas)

TT05_predictions = pd.DataFrame(TT05_predictions, columns = ['prediction_label'])
TT01_predictions = pd.DataFrame(TT01_predictions, columns = ['prediction_label'])
TT10_predictions = pd.DataFrame(TT10_predictions, columns = ['prediction_label'])

In [148]:
TT05_idxf = TT05_datas.index
TT05_p = pd.DataFrame(TT05_idxf, columns = ['index'])
TT05_predictions = pd.concat([TT05_p, TT05_predictions], axis = 1)
TT05_predictions = TT05_predictions.set_index('index')

TT01_idxf = TT01_datas.index
TT01_p = pd.DataFrame(TT01_idxf, columns = ['index'])
TT01_predictions = pd.concat([TT01_p, TT01_predictions], axis = 1)
TT01_predictions = TT01_predictions.set_index('index')

TT10_idxf = TT10_datas.index
TT10_p = pd.DataFrame(TT10_idxf, columns = ['index'])
TT10_predictions = pd.concat([TT10_p, TT10_predictions], axis = 1)
TT10_predictions = TT10_predictions.set_index('index')

In [149]:
final_test = pd.concat([TT05_predictions, TT01_predictions, TT10_predictions])
final_test

,prediction_label
index,
7,0
8,0
14,0
35,0
41,0
...,...
301,1
305,1
307,1


In [150]:
final_test = final_test.sort_index(ascending=True)
final_test['prediction_label']

index
0      1
1      1
2      1
3      0
4      1
      ..
305    1
306    1
307    1
308    1
309    1
Name: prediction_label, Length: 310, dtype: int64

In [151]:
#52
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['Y_Class'] = final_test['prediction_label']
display(sample_submission.head())
sample_submission.to_csv('line3_dropotherstd0_robust_regpyc_svmclass_fill0_submission.csv',index=False)

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,0
4,TEST_004,1


In [152]:
final_test['prediction_label'].value_counts()

1    249
0     34
2     27
Name: prediction_label, dtype: int64

In [153]:
np.array(final_test['prediction_label'])

array([1, 1, 1, 0, 1, 1, 2, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1,
       0, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 0,
       0, 1, 2, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,